In [2]:
%pip install --upgrade git+https://github.com/andrefpf/py-jplm

  Cloning https://github.com/andrefpf/py-jplm to /tmp/pip-req-build-d6ksjire
  Running command git clone --filter=blob:none --quiet https://github.com/andrefpf/py-jplm /tmp/pip-req-build-d6ksjire
  Resolved https://github.com/andrefpf/py-jplm to commit 2640f726ddad0e44802f20c66f40626f1c3c1a2b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


# Configure the encoder

In [81]:
from jplm import JPLMRunner
from jplm.ctc.ctc_configs import (
    CTC_LENSLET_CONFIG,
    CTC_SYNTHETIC_CONFIG,
    CTC_TAROT_CONFIG,
    CTC_SET2_CONFIG,
    CTC_POZNANLAB1_TAU_CONFIG,
)

runner = JPLMRunner("../../parallel-jplm/bin")

In [44]:
import json
from pathlib import Path
from time import time
from collections import defaultdict
        

def nested_dict():
    return defaultdict(nested_dict)

data = nested_dict()

light_field_configs = {
    "Bikes": CTC_LENSLET_CONFIG,
    "Danger_de_Mort": CTC_LENSLET_CONFIG,
    "Fountain_Vincent2": CTC_LENSLET_CONFIG,
    "Stone_Pillars_Outside": CTC_LENSLET_CONFIG,
    "greek": CTC_SYNTHETIC_CONFIG,
    "sideboard": CTC_SYNTHETIC_CONFIG,
    "tarot": CTC_TAROT_CONFIG,
    # "set2": CTC_SET2_CONFIG,
    # "poznanlab1": CTC_POZNANLAB1_TAU_CONFIG,
}

# target_rates = [0.75, 0.1, 0.05, 0.02, 0.01, 0.005, 0.001, 0.0005]
target_rates = [0.001, 0.005, 0.02, 0.1, 0.75]

# algorithms = ["bisection", "hsplit", "hslope"]
algorithms = ["bisection"]

repetitions = 10

base_lf_path = Path("../../../.pgx/")

time_report = nested_dict()

for lf, base_config in light_field_configs.items():
    for tr in target_rates:
        for algorithm in algorithms:
            print(f"Encoding {lf}, {tr}, {algorithm}:")
            config = base_config.copy()
            config["--number-of-threads"] = 16
            config["--target-rate"] = tr
            config["--target-rate-algorithm"] = algorithm
            config["--target-rate-iterations"] = 40 

            input_file = base_lf_path / lf
            times = list() 

            print("\t", end="")
            for i in range(repetitions):
                try:
                    s = time()
                    stdout = runner.encode(input_file, "tmp.jpl", config)
                    e = time()
                    print(i, end=" ")
                    times.append(e-s)
                except Exception as error:
                    print(error)
                    break
            else:
                time_report[lf][tr][algorithm] = times
            print()

        # Save data for checkpoints
        with open(f"time_report_checkpoint_{lf}_{tr}.json", "w") as file:
            json.dump(time_report, file, indent=2)

Encoding Bikes, 0.001, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Bikes, 0.005, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Bikes, 0.02, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Bikes, 0.1, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Bikes, 0.75, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Danger_de_Mort, 0.001, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Danger_de_Mort, 0.005, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Danger_de_Mort, 0.02, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Danger_de_Mort, 0.1, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Danger_de_Mort, 0.75, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Fountain_Vincent2, 0.001, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Fountain_Vincent2, 0.005, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Fountain_Vincent2, 0.02, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Fountain_Vincent2, 0.1, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Fountain_Vincent2, 0.75, bisection:
	0 1 2 3 4 5 6 7 8 9 
Encoding Stone_Pillars_Outside, 0.001, bisection:
	0 1 2 3 4 5 6 7 

In [45]:
time_report_bisection = time_report

In [46]:
import json

with open("time_report.json", "w") as file:
    json.dump(time_report, file, indent=2)

In [86]:
# time report oracle

light_field_configs = {
    # "Bikes": CTC_LENSLET_CONFIG,
    # "Danger_de_Mort": CTC_LENSLET_CONFIG,
    # "Fountain_Vincent2": CTC_LENSLET_CONFIG,
    # "Stone_Pillars_Outside": CTC_LENSLET_CONFIG,
    "greek": CTC_SYNTHETIC_CONFIG,
    "sideboard": CTC_SYNTHETIC_CONFIG,
    "tarot": CTC_TAROT_CONFIG,
}

light_field_lambdas = {
    ("Bikes", 0.001): 3.26894e+07,
    ("Bikes", 0.005): 439381,
    ("Bikes", 0.02):  39521.9,
    ("Bikes", 0.1):   3971.19,
    ("Bikes", 0.75):  263.593,

    ("Danger_de_Mort", 0.001): 4.44621e+07,
    ("Danger_de_Mort", 0.005): 605676,
    ("Danger_de_Mort", 0.02):  65547.3,
    ("Danger_de_Mort", 0.1):   5864.83,
    ("Danger_de_Mort", 0.75):  364.985,

    ("Fountain_Vincent2", 0.001): 4.56958e+07,
    ("Fountain_Vincent2", 0.005): 450320,
    ("Fountain_Vincent2", 0.02):  53230.8,
    ("Fountain_Vincent2", 0.1):   5161.16,
    ("Fountain_Vincent2", 0.75):  321.708,

    ("Stone_Pillars_Outside", 0.001): 3.65863e+07,
    ("Stone_Pillars_Outside", 0.005): 233486,
    ("Stone_Pillars_Outside", 0.02):  27401.8,
    ("Stone_Pillars_Outside", 0.1):   3092.27,
    ("Stone_Pillars_Outside", 0.75):  242.102,

    ("greek", 0.005): 348438,
    ("greek", 0.02):  21271.7,
    ("greek", 0.1):   1321.16,
    ("greek", 0.75):  42.5601,

    ("sideboard", 0.005): 200,
    ("sideboard", 0.02): 200,
    ("sideboard", 0.1): 200,
    ("sideboard", 0.75): 200,

    ("tarot", 0.001): 1.19677e+07,
    ("tarot", 0.005): 774731,
    ("tarot", 0.02):  96624,
    ("tarot", 0.1):   6411.06,
    ("tarot", 0.75):  157.216,
}

target_rates = [0.001, 0.005, 0.02, 0.1, 0.75]

# time_report_oracle = nested_dict()


for lf, base_config in light_field_configs.items():
    for tr in target_rates:
        print(f"Encoding {lf}, {tr}:")
        lambda_ = light_field_lambdas.get((lf, tr))
        
        if lambda_ is None:
            continue
        
        config = base_config.copy()
        config["--number-of-threads"] = 16
        config["--lambda"] = lambda_

        input_file = base_lf_path / lf
        times = list() 

        print("\t", end="")
        for i in range(repetitions):
            try:
                s = time()
                stdout = runner.encode(input_file, f"tmp.jpl", config)
                e = time()
                print(i, end=" ")
                times.append(e-s)
            except Exception as error:
                print(error)
                break
        else:
            time_report_oracle[lf][tr]["oracle"] = times
        print()

    # Save data for checkpoints
    with open(f"time_report_checkpoint_{lf}_{tr}_oracle.json", "w") as file:
        json.dump(time_report, file, indent=2)

Encoding greek, 0.001:
Encoding greek, 0.005:
	0 

KeyboardInterrupt: 

In [17]:
with open("time_report_oracle.json", "w") as file:
        json.dump(time_report_oracle, file, indent=2)

In [ ]:
with open("time_report_combined.json", "w") as file:
        json.dump(time_report, file, indent=2)

In [10]:
import json

with open("time_report.json", "r") as file:
    time_report = json.load(file)

with open("time_report_oracle.json", "r") as file:
    time_report_oracle = json.load(file)

with open("iterations_report.json", "r") as file:
    iterations_report = json.load(file)

In [15]:
from itertools import product


target_rates = [0.001, 0.005, 0.02, 0.1, 0.75]
light_fields = [
    "Bikes",
    "Danger_de_Mort",
    "Fountain_Vincent2",
    "Stone_Pillars_Outside",
    "greek",
    "sideboard",
    "tarot",
]

for lf, tr in product(light_fields, target_rates):
    tr = str(tr)

    try:
        iterations_report[lf][tr]["oracle"] = 
    except Exception as e:
        print(lf, tr)
        # print(e)
        continue

greek 0.001
sideboard 0.001


In [16]:
from pprint import pprint

pprint(time_report)

{'Bikes': {'0.001': {'bisection': [15.829537391662598,
                                   16.101759910583496,
                                   16.283146142959595,
                                   16.1048583984375,
                                   16.083903789520264,
                                   15.756729364395142,
                                   15.791814088821411,
                                   15.921601057052612,
                                   16.072741746902466,
                                   16.28957486152649],
                     'hslope': [6.503457307815552,
                                6.366522789001465,
                                6.068849802017212,
                                5.970118045806885,
                                6.2057201862335205,
                                6.559439659118652,
                                6.750843286514282,
                                6.564558267593384,
                                6.092643022